In [7]:
import pandas as pd
import csv
import os
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
file_path = "C:/Users/Rohan/Downloads/Mock Coding tests result - E&CS.csv"  # Change to your file path
df = pd.read_csv(file_path)
# Remove '@' from the HackerRank ID column
df["HackerRank ID"] = df["HackerRank ID"].astype(str).str.lstrip("@")
# Save the cleaned file
cleaned_file_path = "cleaned_test_2026_csv.csv"
df.to_csv(cleaned_file_path, index=False)
# Get and print the absolute path of the saved file
absolute_path = os.path.abspath(cleaned_file_path)
print(f"Cleaned file saved at: {absolute_path}")


Cleaned file saved at: J:\result-generation\cleaned_test_2026_csv.csv


In [8]:
# Cell 2: Configure Chrome with options to bypass detection
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)  # Prevents auto-close

# Add options to make selenium harder to detect
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Start Selenium
driver = webdriver.Chrome(options=options)

# Execute CDP commands to bypass detection
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    """
})

# Navigate to HackerRank
driver.get("https://www.hackerrank.com/dashboard")

In [10]:
import time
import csv
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


def scrape_leaderboard(driver, leaderboard_url, scraped_file_path):
    wait = WebDriverWait(driver, 15)

    driver.get(leaderboard_url)
    time.sleep(3)

    try:
        dropdown = wait.until(
            EC.element_to_be_clickable((By.ID, "s2id_pagination-length"))
        )
        dropdown.click()
        time.sleep(1)

        option_100 = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//div[@class='select2-result-label' and text()='100']")
            )
        )
        option_100.click()
        time.sleep(2)
    except:
        pass

    data = []
    page = 1

    while True:
        try:
            rows = wait.until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, ".leaderboard-list-view .row")
                )
            )

            for r in rows:
                try:
                    u = r.find_element(By.CSS_SELECTOR, ".span-flex-4 a").text.strip()
                    s = r.find_element(By.CSS_SELECTOR, ".span-flex-3").text.strip()
                    if u and s:
                        data.append([u, s])
                except:
                    continue

        except TimeoutException:
            break

        try:
            nxt = None
            btns = driver.find_elements(By.CSS_SELECTOR, "li a.backbone")
            for b in btns:
                p = b.get_attribute("data-page")
                if p and int(p) == page + 1:
                    nxt = b
                    break

            if not nxt:
                break

            driver.execute_script("arguments[0].scrollIntoView(true);", nxt)
            time.sleep(1)
            nxt.click()
            time.sleep(2)
            page += 1
        except:
            break

    uniq = {}
    for u, s in data:
        if u not in uniq:
            uniq[u] = s

    with open(scraped_file_path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["HackerRank ID", "Score"])
        for u, s in uniq.items():
            w.writerow([u, s])

    return pd.read_csv(scraped_file_path)

leaderboard_url = "https://www.hackerrank.com/contests/tcet-shastra-coding-contest-1-a/leaderboard"
scraped_file_path = "leaderboard_scraped.csv"

scraped_df = scrape_leaderboard(driver, leaderboard_url, scraped_file_path)


In [11]:
scraped_file_path = "leaderboard_scraped.csv"  # First CSV (scraped results)
cleaned_file_path = "cleaned_test_2026_csv.csv"  # Second CSV (cleaned IDs)

scraped_df = pd.read_csv(scraped_file_path)
cleaned_df = pd.read_csv(cleaned_file_path)

merged_df = cleaned_df.merge(scraped_df, on="HackerRank ID", how="left")

final_file_path = "final_shastra_results.csv"
merged_df.to_csv(final_file_path, index=False)

absolute_path = os.path.abspath(final_file_path) 
print(f"Final results saved at: {absolute_path}")

Final results saved at: J:\result-generation\final_shastra_results.csv
